In [1]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

In [2]:
# For some reason i need to run this every time in order to get it work
import os
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk@11/libexec/openjdk.jdk/Contents/Home" 

In [3]:
sc = SparkContext("local[2]", "WordCount")

25/03/25 18:25:19 WARN Utils: Your hostname, ChristoorossAir resolves to a loopback address: 127.0.0.1; using 192.168.1.18 instead (on interface en0)
25/03/25 18:25:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/25 18:25:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/25 18:25:19 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [ ]:
### Creating a stream context with interval 10 seconds
ssc = StreamingContext(sc, 10)

### Getting the lines for each interval, reading from a designated socket
# A server is just a script listening to a particular port, indefinitely running on localhost
# After running this script, i can send data do it from my client (aka local pc) through a 
# command like "cat hamlet.txt | nc -lk localhost 9999"
lines = ssc.socketTextStream('localhost', 9999)  # this is an RDD containing line-object RDDs

/Users/chkapsalis/.pyenv/versions/base/lib/python3.10/site-packages/pyspark/streaming/context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(


-------------------------------------------
Time: 2025-03-25 18:25:30
-------------------------------------------

-------------------------------------------
Time: 2025-03-25 18:25:40
-------------------------------------------

-------------------------------------------
Time: 2025-03-25 18:25:50
-------------------------------------------

-------------------------------------------
Time: 2025-03-25 18:26:00
-------------------------------------------

-------------------------------------------
Time: 2025-03-25 18:26:10
-------------------------------------------

-------------------------------------------
Time: 2025-03-25 18:26:20
-------------------------------------------

-------------------------------------------
Time: 2025-03-25 18:26:40
-------------------------------------------

-------------------------------------------
Time: 2025-03-25 18:26:50
-------------------------------------------

-------------------------------------------
Time: 2025-03-25 18:27:00
----------

In [5]:
### Split all lines into a list of words
words = lines.flatMap(lambda line: line.split(" "))

In [6]:
### Calculate word occurrences in each interval
pairs = words.map(lambda word: (word, 1))
wordCounts = pairs.reduceByKey(lambda x, y: x + y)

In [7]:
### Print calculated counts
wordCounts.pprint()

In [8]:
### In order for the previous processing to occur, we need to execute the following two methods
ssc.start()  # start the computation
ssc.awaitTermination()  # wait for the computation to terminate

ERROR:root:KeyboardInterrupt while sending command.                             
Traceback (most recent call last):
  File "/Users/chkapsalis/.pyenv/versions/base/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/chkapsalis/.pyenv/versions/base/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/chkapsalis/.pyenv/versions/3.10.15/lib/python3.10/socket.py", line 717, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

25/03/25 18:26:30 ERROR Executor: Exception in task 0.0 in stage 25.0 (TID 12)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/chkapsalis/.pyenv/versions/base/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1094, in main
    split_index = read_int(infile)
  File "/Users/chkapsalis/.pyenv/versions/base/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 594, in read_int
    length = stream.read(4)
KeyboardInterrupt

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:572)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:784)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:766)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at sc